<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# Statistique élémentaire avec <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> et  [MLlib](https://spark.apache.org/mllib/)

**Résumé**: Ce tutoriel continue l'initiation à [Spark](https://spark.apache.org/) à l'aide de commandes en Python en utilisant l'API  [`PySpark`](http://spark.apache.org/docs/latest/api/python/); échantillonnage d'une RDD ; présentation de la libriaire MLlib, statistique exploratoire rudimentaire uni et bidimensionnelles, régression logistique.

## 1 Lecture des données
Ce tutoriel s'inspire de ceux proposés par [J. A. Dianes](https://github.com/jadianes/spark-py-notebooks) pour l'utilisation des données du concours [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) concernant près de 9M d'interactions dans un réseau. Elles sont décrites en détail [ici](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names). L'objectif est d'apprendre à détecter des intrusions dans un réseau à partir d'un ensemble de variables ou *features* déjà calculées sur chaque transaction ou ineraction avec le réseau.

Un sous-échantillon est chargé localement avant de créer la RDD.

In [ ]:
sc

In [ ]:
# Chargement du fichier
#Renseignez ici le dossier ou vous souhaitez stocker le fichier téléchargé.
DATA_PATH="/Users/bguillouet/Insa/TP_Insa/data/" 
import urllib.request
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",DATA_PATH+"/kddcup.data_10_percent.gz")
data_file = DATA_PATH+"/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

## 2 Echantillonnage de RDDs
**Attention**, il y a deux opérations disponibles en Spark: la *transformation* `sample` et l'*action* `takeSample`. Il est donc possible de déclarer une séquence de transformations incluant un éhchantillonnage aléatoire simple ou encore d'extraire par une *action* un échantillon d'une RDD qui sera chargée en mémoire avant utilisation par une autre librairie comme par exemple `Scikit-learn` de Pyhton. Bien faire la différence entre *transformation* et *action*

### 2.1 La transformation `sample`

Elle admet jusqu'à trois paramètres, le premier indique si l'échantillonnage est avec remplacement ou non, le deuxième est une fraction d'échantillonnage, le troisième est optionnel pour l'initialisation du générateur (*random seed*). 


In [ ]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)
sample_size = raw_data_sample.count()
total_size = raw_data.count()
print("Sample size is {} of {}".format(sample_size, total_size))

L'intérêt de `sample` est de l'inclure dans une séquence de transformations incluant des opérations (*MapReduce*) d'aggrégation ou de sélections par couples (clef, valeurs).

L'exemple ci-dessous estime, sur un sous-échantillon donc plus rapidement la proportion d'interactions normales.

In [ ]:
from time import time
# transformations à appliquer
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)
# actions + time
t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
tt = time() - t0
# Calcul du taux
sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print("Le taux d interactions normales  est {}".format(round(sample_normal_ratio,3)))
print("Calcul en {} secondes".format(round(tt,3)))

Même chose sans échantillonnage.

In [ ]:
# transformations 
raw_data_items = raw_data.map(lambda x: x.split(","))
normal_tags = raw_data_items.filter(lambda x: "normal." in x)
# actions + time
t0 = time()
normal_tags_count = normal_tags.count()
tt = time() - t0
# Calcul
normal_ratio = normal_tags_count / float(total_size)
print("Le taux d interactions normales  est {}".format(round(normal_ratio,3)) )
print("Calcul en {} secondes".format(round(tt,3)))

### 2.2 L'action `takeSample`
permet d'extraire un échantillon aléatoire simple d'une RDD en le chargeant en mémoire avant utilisation par une librairie hors Spark.

La syntaxe est similaire mais en spécifiant une taille d'échantillon plutôt qu'un taux d'échantillonnage.

In [ ]:
t0 = time()
raw_data_sample = raw_data.takeSample(False, 400000, 1234)
normal_data_sample = [x.split(",") for x in raw_data_sample if "normal." in x]
tt = time() - t0

normal_sample_size = len(normal_data_sample)

normal_ratio = normal_sample_size / 400000.0
print("Le taux d interactions normales  est {}".format(normal_ratio))
print("Calcul en {} secondes".format(round(tt,3)))

Seule la phase d'échantillonnage est distribuée / parallélisée, cette procédure prend plus de temps sur un cluster.

## 3. Présentation de [MLlib](http://spark.apache.org/mllib/)

### 3.1 Préparation des données
Comme cela est déjà largement expliqué dans le [tutoriel](https://github.com/wikistat/Intro-Python) consacré au trafic (munging) des données avec Python `pandas`, leur préparation est une étape essentielle à la qualité des analyses et modélisations qui en découlent. Extraction, filtrage, échantillonnage, complétion des données manquantes, correction, détection d'anomalies ou atypiques, jointures, agrégations ou cumuls, transformations (recodage, discrétisation, réduction, "normalisation"...),  sélection des variables ou *features*, recalages d'images de signaux... sont les principales procédures à mettre en oeuve et de façon itérative avec les étapes d'apprentissage visant les objectifs de l'étude.

Par principe, la plupart de ces étapes, unidimensionnelles, se distribuent naturellement sur les noeuds d'un cluster en exécutant des transformations *MapReduce* et en utilisant les commandes Spark ou des fonctions spécifiques des librairies *MLlib* ou *SparkSQL*.

**N.B.** Il est fréquent, qu'une fois préparées, les données ne soient pas si massives, ou encore il est pertinent d'estimer un modèle sur un échantillon plutôt que sur un corpus très volumineux. Néanmoins, il est important de savoir manipuler des volumes et flux importants, notamment dans l'environnement Spark, avant de passer à la phase d'apprentissage ou modélisation.

### 3.2 Fonctionnalités de [MLlib](http://spark.apache.org/docs/latest/ml-guide.html)
Dans un environnement en pleine évolution, seule la [documentation en ligne](https://spark.apache.org/docs/latest/mllib-guide.html) fait référence. A partir de la version 2.0 de Spark, MLlib évollue en intégrant le type `data frame` de SparkSQL, objet d'un autre tutoriel. MLllib reste néanmoins utilisée pour produire des statistiques élémentaires sur des RDDs. Les traitements plus élaborés d'apprentissage statistique sont détaillés dans les ateliers spécifiques.

Fonctionnalités de MLlib:
- *Statistique de base*: Univariée, corrélation, échantillonnage stratifié, tests d'hypothèse, générateurs aléatoires, transformation (standardisation, quantification de textes avec TF-IDF et vectorisation), sélection (chi2) de variables (*features*).
- *Exploration multidimensionnelle* Classification non-supervisée (k-means avec version en ligne, modèles de mélanges gaussiens, LDA ou *Latent Derichlet Allocation*, réduction de dimension (SVD et ACP mais en java ou scala pas en python), factorisation non négative (NMF) par moindres carrés alternés (ALS).
- *Apprentissage* Méthodes linéaires: SVM, régression gaussienne et binomiale ou logistique avec pénalisation l1 ou l2; estimation par gradient stochastique, ou L-BFGS; classifieur bayésien naïf, arbre de décision, forêts aléatoires, boosting (*gradient boosting machine*).


### 3.3 Types de données
La librairie MLlib manipule exclusivement des RDDs (des data frames avec les développements en cours) de différents types ou classes dont: vecteurs denses ou creux,  `LabeledPOint` pour les algorithmes d'apprentissage ,  `rating` pour les systèmes de recommandation, `Model` pour exploiter un modèle sur des données (prévision). L'arrivée de la classe `DataFrame`modifie en profondeur les usages des types de données. Le développement de cette section reste volontairement limité dans l'attente des nouvelles versions. 
#### Vecteurs denses

In [ ]:
from numpy import array
from pyspark.ml.linalg import Vectors
# vecteur "dense"
# à partir de numpy
denseVec1=array([1.0,0.0,2.0,4.0,0.0])
# en utilisant la classe Vectors
denseVec2=Vectors.dense([1.0,0.0,2.0,4.0,0.0])

#### Vecteurs creux (*sparse*)
Seules les valeurs non nulles sont identifiées et stockées. Il faut préciser la taille du vecteur et les coordonnées de ces valeurs non nulles. C'est défini par un dictionnaire ou par une liste d'indices et de valeurs.

In [ ]:
sparseVec1 = Vectors.sparse(5, {0: 1.0, 2: 2.0, 3: 4.0})
sparseVec2 = Vectors.sparse(5, [0, 2, 3], [1.0, 2.0, 4.0])

#### LabeledPoint
Ce type est spécifique aux algorithmes d'apprentissage et associe un "label", en fait un réel, à un vecteur dense ou creux. Ce "label" est soit la valeur de la variable Y quantitative à modéliser en régression, soit un code de classe: 0.0, 1.0... en classification supervisée ou discrimination. 

#### RDD de vecteurs denses
Données d'interactions représentés par des vecteurs denses à partir du type `array`de *Numpy*.

In [ ]:
# relecture des données
data_file = DATA_PATH+"/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

In [ ]:
import numpy as np

def parse_interaction(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data = raw_data.map(parse_interaction)

## 4. Statistiques élémentaires
### 4.1 Tendances

MLlib propose des statistiques unidimensionnelles par colonne d'un `RDD[Vector]` avec la fonction [`colStats`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.Statistics.colStats) accessible dans [`Statistics`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.Statistics). Cette fonction retourne un [`MultivariateStatisticalSummary`](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.stat.MultivariateStatisticalSummary), qui contient les statistiques *max*, *min*, *moyenne*, *variance*, et *nombre de non nulles*, ainsi que *nombre total*.

In [ ]:
from pyspark.mllib.stat import Statistics
from math import sqrt 

# Compute column summary statistics.
summary = Statistics.colStats(vector_data)

print("Statistique des durées")
print(" Moyenne: {}".format(round(summary.mean()[0],3)))
print(" Ecart type: {}".format(round(sqrt(summary.variance()[0]),3)))
print(" Valeur max: {}".format(round(summary.max()[0],3)))
print(" Valeur min: {}".format(round(summary.min()[0],3)))
print(" Nombre de valeurs: {}".format(summary.count()))
print(" Nombre de valeurs non nulles: {}".format(summary.numNonzeros()[0]))

### 4.2 Pour un *label* donné

In [ ]:
def parse_interaction_with_key(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [1,2,3,41]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return (line_split[41], np.array([float(x) for x in clean_line_split]))

label_vector_data = raw_data.map(parse_interaction_with_key)
# les interactions normales sont filtrées
normal_label_data = label_vector_data.filter(lambda x: x[0]=="normal.")
# Calcul des tendances
normal_summary = Statistics.colStats(normal_label_data.values())
# Affichage
print("Duration Statistics for label: {}".format("normal"))
print(" Mean: {}".format(normal_summary.mean()[0],3))
print(" St. deviation: {}".format(round(sqrt(normal_summary.variance()[0]),3)))
print(" Max value: {}".format(round(normal_summary.max()[0],3)))
print(" Min value: {}".format(round(normal_summary.min()[0],3)))
print(" Total value count: {}".format(normal_summary.count()))
print(" Number of non-zero values: {}".format(normal_summary.numNonzeros()[0]))

### 4.3 Pour un ensemble de  *labels*
Edition d'une fonction avec un label comme paramètre

In [ ]:
def summary_by_label(raw_data, label):
    label_vector_data = raw_data.map(parse_interaction_with_key).filter(lambda x: x[0]==label)
    return Statistics.colStats(label_vector_data.values())

qui redonne les mêmes résultats.

In [ ]:
normal_sum = summary_by_label(raw_data, "normal.")

print("Duration Statistics for label: {}".format("normal"))
print(" Mean: {}".format(normal_sum.mean()[0],3))
print(" St. deviation: {}".format(round(sqrt(normal_sum.variance()[0]),3)))
print(" Max value: {}".format(round(normal_sum.max()[0],3)))
print(" Min value: {}".format(round(normal_sum.min()[0],3)))
print(" Total value count: {}".format(normal_sum.count()))
print(" Number of non-zero values: {}".format(normal_sum.numNonzeros()[0]))

Appliquée à un type d'attaque dont une [liste](http://kdd.ics.uci.edu/databases/kddcup99/training_attack_types) est disponible.

In [ ]:
guess_passwd_summary = summary_by_label(raw_data, "guess_passwd.")

print("Duration Statistics for label: {}".format("guess_password"))
print(" Mean: {}".format(guess_passwd_summary.mean()[0],3))
print(" St. deviation: {}".format(round(sqrt(guess_passwd_summary.variance()[0]),3)))
print(" Max value: {}".format(round(guess_passwd_summary.max()[0],3)))
print(" Min value: {}".format(round(guess_passwd_summary.min()[0],3)))
print(" Total value count: {}".format(guess_passwd_summary.count()))
print(" Number of non-zero values: {}".format(guess_passwd_summary.numNonzeros()[0]))

Durée pour tous les types d'interactions.

In [ ]:
label_list = ["back.","buffer_overflow.","ftp_write.","guess_passwd.",
              "imap.","ipsweep.","land.","loadmodule.","multihop.",
              "neptune.","nmap.","normal.","perl.","phf.","pod.","portsweep.",
              "rootkit.","satan.","smurf.","spy.","teardrop.","warezclient.",
              "warezmaster."]

Statistiques pour chaque type:

In [ ]:
stats_by_label = [(label, summary_by_label(raw_data, label)) for label in label_list]
duration_by_label = [ 
    (stat[0], np.array([float(stat[1].mean()[0]), float(sqrt(stat[1].variance()[0])), float(stat[1].min()[0]), float(stat[1].max()[0]), int(stat[1].count())])) 
    for stat in stats_by_label]

Mise en forme dans un *data frame* de `pandas`.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)
stats_by_label_df = pd.DataFrame.from_items(duration_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')
print("Statistique de la durée par label")
stats_by_label_df

Le tout dans une fonction:

In [ ]:
def get_variable_stats_df(stats_by_label, column_i):
    column_stats_by_label = [
        (stat[0], np.array([float(stat[1].mean()[column_i]), float(sqrt(stat[1].variance()[column_i])), float(stat[1].min()[column_i]), float(stat[1].max()[column_i]), int(stat[1].count())])) 
        for stat in stats_by_label
    ]
    return pd.DataFrame.from_items(column_stats_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')

Qui s'exécute:

In [ ]:
get_variable_stats_df(stats_by_label,0)

Autre exemple:

In [ ]:
print("src_bytes statistics, by label")
get_variable_stats_df(stats_by_label,1)

### 4.4 Corrélations
La fonction `corr` propose des corrélations de Spearman (rangs) ou de Pearson.

In [ ]:
from pyspark.mllib.stat import Statistics 
correlation_matrix = Statistics.corr(vector_data, method="pearson")

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)
col_names = ["duration","src_bytes","dst_bytes","land","wrong_fragment",
             "urgent","hot","num_failed_logins","logged_in","num_compromised",
             "root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds",
             "is_hot_login","is_guest_login","count","srv_count","serror_rate",
             "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

corr_df = pd.DataFrame(correlation_matrix, index=col_names, columns=col_names)
corr_df

Extraction des couples les plus corrélés.

In [ ]:
# Une variable bouléenne est True en cas de forte corrélation
highly_correlated_df = (abs(corr_df) > .8) & (corr_df < 1.0)
# Extraction des noms des variables
correlated_vars_index = (highly_correlated_df==True).any()
correlated_var_names = correlated_vars_index[correlated_vars_index==True].index
highly_correlated_df.loc[correlated_var_names,correlated_var_names]

## 5. Régression logistique

**Remarque**: [J. A. Dianes](https://github.com/jadianes/spark-py-notebooks/blob/master/nb8-mllib-logit/nb8-mllib-logit.ipynb) utilise la procédure ci-dessus (variables les plus corrélées) ou des tests du Chi2 pour sélectionner des variables avant d'estimer une régression (logistique). C'est contraitre aux usages en Statistique qui privilégient des méthodes *pas-à-pas* (*forward, backward, both*) en minimisant un critère comme AIC (cf. un tutoriel en R) ou la prise en compte d'une *pénalisation lasso*. C'est ce dernier cas qui est privilégié, tant dans la librairie *Scikit-learn* de Python, que dans *Mllib*.

Pour la suite,  l'ensemble des données dela [KDD cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) sont utilisées. Il peut être prudent de *redémarer le noyau* afin de libérer la mémoire avant de poursuivre.

### 5.1 Lecture et préparation des données

In [ ]:
# Données d'apprentissage
import urllib.request
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz",DATA_PATH+"/kddcup.data.gz")
data_file = DATA_PATH+"/kddcup.data.gz"
raw_data = sc.textFile(data_file)
print("Train data size is {}".format(raw_data.count()))

In [ ]:
# Données de test
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz", DATA_PATH+"/corrected.gz")
test_data_file = DATA_PATH+"/corrected.gz"
test_raw_data = sc.textFile(test_data_file)

print("Test data size is {}".format(test_raw_data.count()))

**Labeled Points** est le format à utiliser pour un objectif d'apprentissage supervisé. Le *label* contient la variable à modéliser, classe (entier de 0 à nombre de classes - 1) ou valeur quantitative.

Il s'agit de modéliser / prévoir l'occurence d'une attaque indépendamment du type de celle-ci. Il s'agit donc d'une variable binaire (0,1) à construire. 

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array
# fonction à appliquer à chaque ligne ou interaction sur le réseau
def parse_interaction(line):
    line_split = line.split(",")
    # supprime les colonnes [1,2,3,41]
    clean_line_split = line_split[0:1]+line_split[4:41]
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))
# exécution sur données d'apprentissage
training_data = raw_data.map(parse_interaction)

In [ ]:
# Données de test
test_data = test_raw_data.map(parse_interaction)

### 5.2 Estimation de la [régression logistique](http://wikistat.fr/pdf/st-m-app-rlogit.pdf)
C'est la procédure classique pour prévoir une variable binaire. Mllib propose deux [algorithmes](https://spark.apache.org/docs/latest/mllib-linear-methods.html#logistic-regression) pour l'optimisation (*mini-batch gradient descent* et L-BFGS). L-BFGS converge en principe plus vite. 

Comme dans *Scikit-learn* la procédure propose d'inclure une régularisation par pénalisation *l2* (ridge) ou *l1* (lasso). Le paramètre `regType` précise le type (`l1` ou `l2`) tandis que `regParam` précise la pénalisation. Par défaut, il n'y a pas de régularisation.

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
LogisticRegressionWithLBFGS?

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
t0 = time()
logit_model = LogisticRegressionWithLBFGS.train(training_data)
tt = time() - t0
print("Apprentissage en {} seconds".format(round(tt,3)))

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile("sample_svm_data.txt")
parsedData = data.map(parsePoint)

# Build the model
model = LogisticRegressionWithLBFGS.train(parsedData)

### 5.3 Estimation de l'erreur
Une fonction *map* permet de calculer la prévision de chaque observation du test.

In [ ]:
labels_and_preds = test_data.map(lambda p: (p.label, logit_model.predict(p.features)))

Il suffit ensuite de calculer l'erreur de prévision sur l'échantillon test. 

In [ ]:
t0 = time()
test_accuracy = labels_and_preds.filter(lambda (v, p): v == p).count() / float(test_data.count())
errreur=1-test_accuracy
tt = time() - t0
print("Calcul en {} secondes. Le taux d'erreur est {}".format(round(tt,3), round(erreur,4)))

**Remarques**; **exercices**

- Les variables qualitatives `protocol` et `service`ont été éliminées par simplicité, il faudrait les ajouter sous la forme d'indicatrices (*dummy variables*)
- Introduire une pénalisation lasso, [optimiser](https://spark.apache.org/docs/latest/ml-tuning.html) le paramètre par validation croisée. Le résultat est-il meilleure?
- [J.A. Dianes](https://github.com/jadianes/spark-py-notebooks/blob/master/nb9-mllib-trees/nb9-mllib-trees.ipynb) estime également un [arbre binaire de décision](https://spark.apache.org/docs/latest/mllib-decision-tree.html) sur ces données. malheureusement, comme dans le cas de la librairie *Scikit-learn*, il n'est pas possible d'optimiser correctement l'élagage d'un arbre proposé par *Mllib*. Il se contente de construire un arbre de profondeur maximum 3, donc facile à interpréter, avec une erreur de prévision à peine supérieure à celle de la régression logistique.
- Utiliser les [forêts aléatoires](https://spark.apache.org/docs/latest/mllib-ensembles.html#random-forests) sur ces données.

Ces méthodes plus sophistiquées d'apprentissage sont testées dans les différents cas d'usage.